# Market Impact Function Modeling

This notebook explores and models the temporary market impact function g_t(x) based on historical limit order book data.

## Objectives:
1. Load and analyze order book data
2. Estimate slippage for different trade sizes
3. Fit linear and nonlinear impact models
4. Compare model performance visually

## Models:
- **Linear Model**: g_t(x) = β_t * x
- **Nonlinear Model**: g_t(x) = α_t * x² + β_t * x

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Import our custom modules
import sys
sys.path.append('../src')
from models.impact import ImpactModel

print("Libraries imported successfully!")

## 1. Generate Synthetic Order Book Data

Since we don't have real order book data, we'll generate synthetic data that mimics realistic market microstructure.

In [ ]:
# Initialize impact model and generate synthetic order book data
impact_model = ImpactModel(model_type='linear')

# Generate synthetic order book data for 390 time intervals (6.5 hours)
orderbook_data = impact_model.generate_synthetic_orderbook(n_snapshots=390, n_levels=20)

print(f"Generated order book data with {len(orderbook_data)} records")
print(f"Time range: {orderbook_data['time'].min()} to {orderbook_data['time'].max()}")
print(f"\nFirst few records:")
orderbook_data.head(10)

## 2. Visualize Order Book Depth

Let's examine the price levels vs. volume (depth) for a few time snapshots.

In [ ]:
# Plot order book depth for selected time points
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

time_points = [0, 50, 100, 200, 300, 389]  # Sample time points

for i, t in enumerate(time_points):
    # Get data for time t
    snapshot = orderbook_data[orderbook_data['time'] == t]
    
    # Separate bids and asks
    bids = snapshot[snapshot['side'] == 'bid'].sort_values('price', ascending=False)
    asks = snapshot[snapshot['side'] == 'ask'].sort_values('price')
    
    # Calculate cumulative volumes
    bids['cum_volume'] = bids['size'].cumsum()
    asks['cum_volume'] = asks['size'].cumsum()
    
    # Plot depth chart
    axes[i].step(bids['cum_volume'], bids['price'], where='post', color='green', linewidth=2, label='Bids')
    axes[i].step(asks['cum_volume'], asks['price'], where='post', color='red', linewidth=2, label='Asks')
    
    axes[i].set_title(f'Order Book Depth at t={t}')
    axes[i].set_xlabel('Cumulative Volume')
    axes[i].set_ylabel('Price')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Estimate Slippage for Different Trade Sizes

We'll estimate the market impact (slippage) for various trade sizes using the order book depth.

In [ ]:
# Estimate slippage for different trade sizes
print("Estimating slippage for different trade sizes...")

# Define trade sizes to analyze (from 100 to 50,000 shares)
trade_sizes = np.logspace(2, 4.7, 30)  # 30 points from 100 to ~50,000

# Estimate slippage
slippage_data = impact_model.estimate_slippage(orderbook_data, trade_sizes)

print(f"Generated slippage estimates for {len(slippage_data)} combinations")
print(f"Trade size range: {trade_sizes.min():.0f} to {trade_sizes.max():.0f} shares")
print(f"\nSlippage data sample:")
slippage_data.head(10)

In [ ]:
# Visualize slippage patterns
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot 1: Slippage vs Trade Size for selected time points
selected_times = [0, 100, 200, 300]
for t in selected_times:
    time_data = slippage_data[slippage_data['time'] == t]
    axes[0].plot(time_data['trade_size'], time_data['slippage'], 
                marker='o', markersize=4, label=f't={t}')

axes[0].set_xlabel('Trade Size (shares)')
axes[0].set_ylabel('Slippage')
axes[0].set_title('Slippage vs Trade Size')
axes[0].set_xscale('log')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Slippage evolution over time for fixed trade size
fixed_size = 5000  # 5000 shares
size_data = slippage_data[np.abs(slippage_data['trade_size'] - fixed_size) < 500]
time_slippage = size_data.groupby('time')['slippage'].mean().reset_index()

axes[1].plot(time_slippage['time'], time_slippage['slippage'], 'b-', linewidth=2)
axes[1].set_xlabel('Time (minutes)')
axes[1].set_ylabel('Slippage')
axes[1].set_title(f'Slippage Over Time (Trade Size ≈ {fixed_size} shares)')
axes[1].grid(True, alpha=0.3)

# Plot 3: Heatmap of slippage
# Create pivot table for heatmap
pivot_data = slippage_data.pivot_table(index='trade_size', columns='time', values='slippage')
# Subsample for visualization
pivot_subset = pivot_data.iloc[::3, ::30]  # Every 3rd trade size, every 30th time

im = axes[2].imshow(pivot_subset.values, aspect='auto', cmap='viridis', origin='lower')
axes[2].set_title('Slippage Heatmap')
axes[2].set_xlabel('Time')
axes[2].set_ylabel('Trade Size')
plt.colorbar(im, ax=axes[2], label='Slippage')

plt.tight_layout()
plt.show()

## 4. Fit Linear Impact Model

First, let's fit the linear model: g_t(x) = β_t * x

In [ ]:
# Fit linear impact model
print("Fitting linear impact model: g_t(x) = β_t * x")

linear_model = ImpactModel(model_type='linear')
linear_parameters = linear_model.fit_impact_model(slippage_data)

print(f"Fitted linear models for {len(linear_parameters)} time points")

# Get model summary
linear_summary = linear_model.get_model_summary()
print(f"\nLinear model parameters (β_t):")
print(f"Mean β: {linear_summary['beta'].mean():.8f}")
print(f"Std β: {linear_summary['beta'].std():.8f}")
print(f"Min β: {linear_summary['beta'].min():.8f}")
print(f"Max β: {linear_summary['beta'].max():.8f}")

linear_summary.head(10)

In [ ]:
# Visualize linear model parameters
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plot beta coefficients over time
axes[0].plot(linear_summary['time'], linear_summary['beta'], 'b-', linewidth=2)
axes[0].set_xlabel('Time (minutes)')
axes[0].set_ylabel('β_t (Linear Coefficient)')
axes[0].set_title('Linear Impact Coefficient Over Time')
axes[0].grid(True, alpha=0.3)

# Plot histogram of beta values
axes[1].hist(linear_summary['beta'], bins=30, alpha=0.7, color='blue', edgecolor='black')
axes[1].axvline(linear_summary['beta'].mean(), color='red', linestyle='--', 
               linewidth=2, label=f'Mean: {linear_summary["beta"].mean():.8f}')
axes[1].set_xlabel('β_t Value')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Linear Coefficients')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Fit Nonlinear Impact Model

Now let's fit the nonlinear model: g_t(x) = α_t * x² + β_t * x

In [ ]:
# Fit nonlinear impact model
print("Fitting nonlinear impact model: g_t(x) = α_t * x² + β_t * x")

nonlinear_model = ImpactModel(model_type='nonlinear')
nonlinear_parameters = nonlinear_model.fit_impact_model(slippage_data)

print(f"Fitted nonlinear models for {len(nonlinear_parameters)} time points")

# Get model summary
nonlinear_summary = nonlinear_model.get_model_summary()
print(f"\nNonlinear model parameters:")
print(f"Mean α: {nonlinear_summary['alpha'].mean():.10f}")
print(f"Mean β: {nonlinear_summary['beta'].mean():.8f}")
print(f"Std α: {nonlinear_summary['alpha'].std():.10f}")
print(f"Std β: {nonlinear_summary['beta'].std():.8f}")

nonlinear_summary.head(10)

In [ ]:
# Visualize nonlinear model parameters
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot alpha coefficients over time
axes[0, 0].plot(nonlinear_summary['time'], nonlinear_summary['alpha'], 'r-', linewidth=2)
axes[0, 0].set_xlabel('Time (minutes)')
axes[0, 0].set_ylabel('α_t (Quadratic Coefficient)')
axes[0, 0].set_title('Quadratic Impact Coefficient Over Time')
axes[0, 0].grid(True, alpha=0.3)

# Plot beta coefficients over time
axes[0, 1].plot(nonlinear_summary['time'], nonlinear_summary['beta'], 'b-', linewidth=2)
axes[0, 1].set_xlabel('Time (minutes)')
axes[0, 1].set_ylabel('β_t (Linear Coefficient)')
axes[0, 1].set_title('Linear Impact Coefficient Over Time')
axes[0, 1].grid(True, alpha=0.3)

# Plot histogram of alpha values
axes[1, 0].hist(nonlinear_summary['alpha'], bins=30, alpha=0.7, color='red', edgecolor='black')
axes[1, 0].axvline(nonlinear_summary['alpha'].mean(), color='blue', linestyle='--', 
                  linewidth=2, label=f'Mean: {nonlinear_summary["alpha"].mean():.10f}')
axes[1, 0].set_xlabel('α_t Value')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Distribution of Quadratic Coefficients')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot histogram of beta values
axes[1, 1].hist(nonlinear_summary['beta'], bins=30, alpha=0.7, color='blue', edgecolor='black')
axes[1, 1].axvline(nonlinear_summary['beta'].mean(), color='red', linestyle='--', 
                  linewidth=2, label=f'Mean: {nonlinear_summary["beta"].mean():.8f}')
axes[1, 1].set_xlabel('β_t Value')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Distribution of Linear Coefficients')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Compare Linear and Nonlinear Models

Let's visually compare the fitted impact functions for both models.

In [ ]:
# Compare model predictions for selected time points
comparison_times = [0, 100, 200, 300, 389]
trade_size_range = np.linspace(100, 20000, 100)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, t in enumerate(comparison_times):
    if t in linear_model.fitted_models and t in nonlinear_model.fitted_models:
        # Get predictions from both models
        linear_predictions = [linear_model.predict_impact(t, size) for size in trade_size_range]
        nonlinear_predictions = [nonlinear_model.predict_impact(t, size) for size in trade_size_range]
        
        # Plot both models
        axes[i].plot(trade_size_range, linear_predictions, 'b-', linewidth=2, label='Linear Model')
        axes[i].plot(trade_size_range, nonlinear_predictions, 'r-', linewidth=2, label='Nonlinear Model')
        
        # Plot actual data points for this time
        actual_data = slippage_data[slippage_data['time'] == t]
        axes[i].scatter(actual_data['trade_size'], actual_data['slippage'], 
                       alpha=0.6, color='green', s=20, label='Actual Data')
        
        axes[i].set_title(f'Impact Function Comparison at t={t}')
        axes[i].set_xlabel('Trade Size (shares)')
        axes[i].set_ylabel('Market Impact')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)

# Remove empty subplot
if len(comparison_times) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

## 7. Model Performance Analysis

Let's evaluate how well each model fits the data using R-squared and RMSE metrics.

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

# Calculate model performance metrics
performance_data = []

for t in range(min(50, len(linear_model.fitted_models))):
    if t in linear_model.fitted_models and t in nonlinear_model.fitted_models:
        # Get actual data for this time
        time_data = slippage_data[slippage_data['time'] == t]
        
        if len(time_data) > 0:
            actual_impact = time_data['slippage'].values
            trade_sizes = time_data['trade_size'].values
            
            # Get model predictions
            linear_pred = [linear_model.predict_impact(t, size) for size in trade_sizes]
            nonlinear_pred = [nonlinear_model.predict_impact(t, size) for size in trade_sizes]
            
            # Calculate metrics
            linear_r2 = r2_score(actual_impact, linear_pred)
            nonlinear_r2 = r2_score(actual_impact, nonlinear_pred)
            
            linear_rmse = np.sqrt(mean_squared_error(actual_impact, linear_pred))
            nonlinear_rmse = np.sqrt(mean_squared_error(actual_impact, nonlinear_pred))
            
            performance_data.append({
                'time': t,
                'linear_r2': linear_r2,
                'nonlinear_r2': nonlinear_r2,
                'linear_rmse': linear_rmse,
                'nonlinear_rmse': nonlinear_rmse
            })

performance_df = pd.DataFrame(performance_data)

print("Model Performance Comparison:")
print(f"Average Linear R²: {performance_df['linear_r2'].mean():.4f}")
print(f"Average Nonlinear R²: {performance_df['nonlinear_r2'].mean():.4f}")
print(f"Average Linear RMSE: {performance_df['linear_rmse'].mean():.6f}")
print(f"Average Nonlinear RMSE: {performance_df['nonlinear_rmse'].mean():.6f}")

performance_df.head()

In [ ]:
# Plot performance comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# R-squared comparison
axes[0].plot(performance_df['time'], performance_df['linear_r2'], 'b-', linewidth=2, label='Linear Model')
axes[0].plot(performance_df['time'], performance_df['nonlinear_r2'], 'r-', linewidth=2, label='Nonlinear Model')
axes[0].set_xlabel('Time (minutes)')
axes[0].set_ylabel('R² Score')
axes[0].set_title('Model R² Score Over Time')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# RMSE comparison
axes[1].plot(performance_df['time'], performance_df['linear_rmse'], 'b-', linewidth=2, label='Linear Model')
axes[1].plot(performance_df['time'], performance_df['nonlinear_rmse'], 'r-', linewidth=2, label='Nonlinear Model')
axes[1].set_xlabel('Time (minutes)')
axes[1].set_ylabel('RMSE')
axes[1].set_title('Model RMSE Over Time')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Save Models and Data

Let's save our fitted models and processed data for use in the trade allocation optimization notebook.

In [ ]:
import pickle

# Save fitted models
models_to_save = {
    'linear_model': linear_model,
    'nonlinear_model': nonlinear_model,
    'orderbook_data': orderbook_data,
    'slippage_data': slippage_data,
    'performance_data': performance_df
}

with open('../data/fitted_impact_models.pkl', 'wb') as f:
    pickle.dump(models_to_save, f)

# Also save as CSV for external use
linear_summary.to_csv('../data/linear_model_parameters.csv', index=False)
nonlinear_summary.to_csv('../data/nonlinear_model_parameters.csv', index=False)
slippage_data.to_csv('../data/slippage_estimates.csv', index=False)

print("Models and data saved successfully!")
print("Files saved:")
print("- ../data/fitted_impact_models.pkl")
print("- ../data/linear_model_parameters.csv")
print("- ../data/nonlinear_model_parameters.csv")
print("- ../data/slippage_estimates.csv")

## Summary

In this notebook, we have:

1. **Generated synthetic order book data** with realistic market microstructure characteristics
2. **Analyzed order book depth** and visualized price-volume relationships
3. **Estimated market impact (slippage)** for various trade sizes across time
4. **Fitted two impact models**:
   - Linear: g_t(x) = β_t * x
   - Nonlinear: g_t(x) = α_t * x² + β_t * x
5. **Compared model performance** using R² and RMSE metrics
6. **Saved fitted models** for use in trade allocation optimization

### Key Findings:

- The market impact increases with trade size, as expected
- Impact coefficients vary over time, reflecting changing market conditions
- Both models capture the general relationship, with the nonlinear model providing more flexibility
- The fitted models are ready for use in optimizing trade allocation strategies

**Next Steps**: Use these impact models in the trade allocation optimization notebook to find the optimal execution strategy.